## A simple model for demand and supply of publicly-provided services in a city

In [1]:
import geopy
import numpy as np
import pandas as pd
from sklearn import gaussian_process
from matplotlib import pyplot as plt 
from enum import Enum


In [7]:
gaussKern = gaussian_process.kernels.RBF
get_random_pos = lambda n: np.random.uniform(low=[45.40,9.1],high=[45.50, 9.3], size=(n,2))

In [31]:
## Enum classes
class AgeGroup(Enum):
    Newborn= (0, 3)
    Child= (4,15)
    Young= (16,25)
    Junior= (26,35)
    Senior= (36, 50)
    Over50= (50, 64)
    Over65= (66, 80)
    Over80= (81, 200)
    def __init__(self, startAge, endAge):
        self.start = startAge
        self.end = endAge
    
    @staticmethod
    def all():
        return([g for g in AgeGroup])
    @property
    def range(self): return self.end - self.start
    
class SummaryNorm(Enum):
    l1= lambda x: sum(abs(x))
    l2= lambda x: (sum(x**2))**0.5
    lInf= lambda x: max(x)
    

class ServiceType(Enum):
    School = (1, SummaryNorm.l2)
    Football = (2, SummaryNorm.l2)
    SocialSupport = (3, SummaryNorm.l2)
    PoliceStation = (4, SummaryNorm.l2)
    #etc
    def __init__(self, _, aggrNormInput=SummaryNorm.l2):
        self.aggrNorm = aggrNormInput
        # initialise demand factors for each age group
        self.demandFactors = {a: 1 for a in AgeGroup.all()}
    
    def aggregate_units(self, x, axis=1):
        # assumes position are stacked in rows
        return np.apply_along_axis(self.aggrNorm, axis, x)
    
    @staticmethod
    def all():
        return([g for g in ServiceType])
    
#demandFactors = pd.DataFrame(np.ones([len(AgeGroup.all()), len(ServiceType.all())]), 
#                             index=AgeGroup.all(), columns=ServiceType.all())

In [4]:
## ServiceUnit class
class ServiceUnit:
    def __init__(self, service, name='', position=(45.4641, 9.1919), users=AgeGroup.all(), times=[], scale=1):
        assert isinstance(position, tuple) & (len(position) == 2), 'Position must be a pair tuple' #will be replaced by nicer class
        assert isinstance(service, ServiceType), 'Service must belong to the Enum'
        self.name = name
        self.service = service
        self.site = position  # a Service can have many sites, and a site is not uniquely assigned to a service
        self.times = times
        
        # how the service availablity area varies for different age groups
        kPropagationTest = 3
        self.propagation ={g: (0.04 + .001*np.round(np.random.normal(),2))*kPropagationTest*scale for g in users} 
        self.kernel = {g: gaussKern(length_scale=l) for g, l in self.propagation.items()}
        
    def evaluate(self, targetPositions, ageGroup):
        assert targetPositions.shape[1] == 2, '2D targets expected'
        reshapedPos = np.array(self.site).reshape(-1,2)
        # evaluate kernel to get level service score. If age group is not served, return 0 as default
        if self.kernel.__contains__(ageGroup):
            score = self.kernel[ageGroup](targetPositions, reshapedPos)
        else:
            score = 0
        return np.squeeze(score)
    
    @property
    def users(self): return list(self.propagation.keys())

In [5]:
### Supply modelling
def evaluate_services_at(positions, unitsList, outputServices= [t for t in ServiceType]):
    # set all age groups as output default
    outputAgeGroups = AgeGroup.all()

    # for a given position, output is a (ageGroup VS serviceType) table. Let's initialise it
    outScores = {(a, t): np.zeros(positions.shape[0]) for a in outputAgeGroups for t in outputServices} 
    for thisServType in outputServices:
        for thisAgeGroup in outputAgeGroups:
            serviceUnits = [u for u in unitsList if u.service == thisServType]
            if not serviceUnits:
                continue
            unitValues = np.stack(list(map(lambda x: x.evaluate(positions, thisAgeGroup), serviceUnits)),
                                 axis=-1)
            # aggregate unit contributions according to the service type norm
            outScores[(thisAgeGroup, thisServType)] = thisServType.aggregate_units(unitValues)
    return outScores
        

In [8]:
test = [ServiceUnit(ServiceType.PoliceStation, 'Duomo'), 
        ServiceUnit(ServiceType.PoliceStation, 'Ripamonti', position=(45.43, 9.201))]
evaluate_services_at(get_random_pos(50), test)


{(<AgeGroup.Child: (4, 15)>,
  <ServiceType.Football: (2, <function SummaryNorm.<lambda> at 0x7f03977980d0>)>): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 (<AgeGroup.Child: (4, 15)>,
  <ServiceType.PoliceStation: (4, <function SummaryNorm.<lambda> at 0x7f03977980d0>)>): array([1.09496195, 0.9628518 , 1.36036626, 1.31351009, 1.31397537,
        1.05380076, 0.99016339, 1.13689954, 1.25984922, 1.06866896,
        1.30368848, 1.30649697, 1.37424469, 1.20541246, 1.10148779,
        1.30616328, 1.09151981, 1.26618662, 1.22609834, 1.20800307,
        1.14912205, 1.31843586, 1.30889724, 1.37431641, 1.06056142,
        1.39674326, 1.30385225, 1.12533765, 1.34625474, 1.20341252,
        1.045031  , 1.11647047, 1.12578908, 1.38678604, 1.27630774,
        1.39303456, 1.36033552, 1.22152391, 1.06997041, 0.98474303,
   

In [33]:
### Demand modelling
class Household:
    def __init__(self, position=get_random_pos(1), members=None):
        if not members: members = {a: 1 for a in AgeGroup.all()}
        self.position = position
        self.members = members
        
def evaluate_demand(householdList, outputServices= [t for t in ServiceType]):
    # export demand for positions, agegroups and services 
    pass

In [15]:
zz = Household()
zz.members

{<AgeGroup.Child: (4, 15)>: 1,
 <AgeGroup.Junior: (26, 35)>: 1,
 <AgeGroup.Newborn: (0, 3)>: 1,
 <AgeGroup.Over50: (50, 64)>: 1,
 <AgeGroup.Over65: (66, 80)>: 1,
 <AgeGroup.Over80: (81, 200)>: 1,
 <AgeGroup.Senior: (36, 50)>: 1,
 <AgeGroup.Young: (16, 25)>: 1}